In [ ]:
from crossword_solver.crossword_detection import detect_crossword_from_file
from crossword_solver.solver_utils import find_whole_crossword_candidates
import matplotlib.pyplot as plt
from crossword_solver.paths import DATA_FOLDER_PATH
import os
from crossword_solver.crossword_solver import solve_crossword

In [ ]:
import cv2
from crossword_solver.square_classification import SquareType
from copy import deepcopy
from PIL import ImageFont, ImageDraw, Image
from crossword_solver.paths import FONT_PATH
from io import BytesIO
from base64 import b64encode

def show_image(image):
    correct_colors = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    plt.imshow(correct_colors)
    plt.show()


def plot_square_types(crossword):
    font = cv2.FONT_HERSHEY_SIMPLEX   
    fontScale = 1   
    color = (0, 255, 0) 
    thickness = 2
    
    type_to_text = {
        SquareType.HINT : "V",
        SquareType.MULTIHINT : "W",
        SquareType.IRRELEVANT : "X",
        SquareType.EMPTY : "0",
    }

    plotting_image = deepcopy(crossword.image)
    for gs in crossword.grid.flatten():
        t = type_to_text[gs.type]
        plotting_image = cv2.circle(plotting_image, (gs.x_min,gs.y_min), radius=3, color=(0, 0, 0), thickness=-1)
        plotting_image = cv2.putText(plotting_image, t, (gs.x_min+20,gs.y_max-20), font,  fontScale, color, thickness, cv2.LINE_AA)
    return plotting_image
    #return cv2_to_base64(plotting_image) 

def plot_contours(crossword):
    color = (0, 255, 0) 
    thickness = 2

    plotting_image = deepcopy(crossword.image)
    for gs in crossword.grid.flatten():
        plotting_image = cv2.line(plotting_image, (gs.x_min,gs.y_min), (gs.x_max,gs.y_min),
                                  color=color, thickness=thickness)
        plotting_image = cv2.line(plotting_image, (gs.x_max,gs.y_min), (gs.x_max,gs.y_max),
                                  color=color, thickness=thickness)
        plotting_image = cv2.line(plotting_image, (gs.x_max,gs.y_max), (gs.x_min,gs.y_max),
                                  color=color, thickness=thickness)
        plotting_image = cv2.line(plotting_image, (gs.x_min,gs.y_max), (gs.x_min,gs.y_min),
                                  color=color, thickness=thickness)
    return plotting_image   

def plot_solution_texts(crossword, solution_matrix):
    font = ImageFont.truetype(FONT_PATH, 25)
    image = deepcopy(crossword.image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image)    
    for gs in crossword.grid.flatten():
        if gs.type == SquareType.EMPTY:
            t = solution_matrix[gs.grid_x, gs.grid_y].upper()
            if t != '_':
                draw = ImageDraw.Draw(pil_image)
                draw.text((gs.x_min+20,gs.y_max-45), t, font=font, fill='#000')
    return pil_image
    #return pil_to_base64(pil_image)

def plot_no_solution(crossword):
    font = ImageFont.truetype(FONT_PATH, 40)
    image = deepcopy(crossword.image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image)    
    t = "LAHENDUSI EI LEITUD"
    draw = ImageDraw.Draw(pil_image)
    draw.text((crossword.grid_x+100, crossword.grid_y+100), t, font=font, fill='#cd0000')
    return pil_image
    #return pil_to_base64(pil_image)

def save_image(image, name):
    cv2.imwrite(f'C:/Users/ianigol/Documents/MAGISTER/ristsonalahendaja/crossword_solver/data/pictures/{name}.png', image)

In [ ]:
nr = 10
path = os.path.join(DATA_FOLDER_PATH, 'pictures', f'ristsona_{nr}.png')
crossword = detect_crossword_from_file(path)

In [ ]:
show_image(plot_contours(crossword))

In [ ]:
save_image(plot_contours(crossword), f'ristsona_{nr}_kontuurid')

In [ ]:
find_whole_crossword_candidates(crossword)

In [ ]:
sqtypes = plot_square_types(crossword)

In [ ]:
name = f'ristsona_{nr}_kategooriad'
save_image(sqtypes, name)

In [ ]:
no_candidates_counter = 0
for h in crossword.hints:
    print('vihje:', h.hint)
    if len(h.candidates) > 0:
        for t in h.candidates:
            print(t.text, f'({t.source})')
    else:
        print('-')
        no_candidates_counter+=1
    print()
    print('_________________')

In [ ]:
print(len(crossword.hints), no_candidates_counter)

In [ ]:
sum_found_candidates = 0
for h in crossword.hints:
    sum_found_candidates += len(h.candidates)
sum_found_candidates/len(crossword.hints)

In [ ]:
results = solve_crossword(crossword)
topn_results = sorted(results, key=lambda x: x[2], reverse = True)[:5]
topn_results = [(plot_solution_texts(crossword, matrix), round(score), intersections) for matrix, score, intersections in topn_results]

In [ ]:
results